In [2]:
##test
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive
import joblib

def compute_esi(row):
    # Avoid division by zero
    radius_score = (1 - abs(row['radius'] - 1) / (row['radius'] + 1)) ** 0.57
    mass_score = (1 - abs(row['mass'] - 1) / (row['mass'] + 1)) ** 1.07
    temp_score = (1 - abs(row['temperature'] - 288) / (row['temperature'] + 288)) ** 5.58

    # You can optionally add flux_score with a similar formula

    esi = radius_score * mass_score * temp_score
    return round(esi * 100, 2)  # Return as percentage

# live data from the NASA Exoplanet Archive
data = NasaExoplanetArchive.query_criteria(
    table="ps",
    select="pl_name, pl_bmasse, pl_rade, pl_eqt, pl_insol",
    where="pl_bmasse IS NOT NULL AND pl_rade IS NOT NULL AND pl_eqt IS NOT NULL AND pl_insol IS NOT NULL"
)

# renaming the columns for clarity
g = data.to_pandas()
g.rename(columns={
    'pl_name': 'name',
    'pl_bmasse': 'mass',
    'pl_rade': 'radius',
    'pl_eqt': 'temperature',
    'pl_insol': 'flux'
}, inplace=True)


# Dropping rows with missing values in key columns
g.dropna(subset=['mass', 'radius', 'temperature', 'flux'], inplace=True)

# Loading the previously trained model and scaler to predict habitability
scaler = joblib.load('scaler.pkl')
model = joblib.load('habitability_model.pkl')

# Sclaing the features for prediction
features = ['mass', 'radius', 'temperature', 'flux']
X_new = scaler.transform(g[features])

g['potentially_habitable'] = model.predict(X_new)

# Filtering the potentially habitable planets
habitable_planets = g[g['potentially_habitable'] == 1]
habitable_planets['ESI (%)'] = habitable_planets.apply(compute_esi, axis=1)
habitable_planets.to_csv('predicted_habitable_exoplanets.csv', index=False)


# Those planets are then expored to a CSV file
habitable_planets.to_csv('predicted_habitable_exoplanets.csv', index=False)

# Displaying the results
print("Total exoplanets analyzed:", len(g))
print("Potentially habitable exoplanets found:", len(habitable_planets))
print("List of habitable planets found:")
print(habitable_planets[['name', 'mass', 'radius', 'temperature', 'flux']].head(len(habitable_planets)))

Total exoplanets analyzed: 572
Potentially habitable exoplanets found: 17
List of habitable planets found:
                 name         mass     radius  temperature   flux
2         Kepler-22 b     9.100000   2.100000        279.0  1.013
29       Kepler-553 c  2129.450327  11.578877        251.0  0.589
43              PH2 b    87.084984   9.337081        295.1  1.200
105            K2-3 d     2.200000   1.458000        305.2  1.440
109           K2-18 b     8.630000   2.610000        254.9  1.005
143      TRAPPIST-1 f     0.680000   1.045000        219.0  0.382
144      TRAPPIST-1 g     1.340000   1.127000        198.6  0.258
148        LHS 1140 b     5.600000   1.730000        226.0  0.430
149        LHS 1140 b     6.980000   1.727000        235.0  0.503
150        LHS 1140 b     6.650000   1.430000        230.0  0.460
181     Kepler-1654 b   158.915000   9.180171        206.0  0.300
228  EPIC 248847494 b  4131.790000  12.441990        183.0  0.190
246     Kepler-1661 b    17.000000 

C:\Users\kisho\AppData\Local\Temp\ipykernel_33668\1273398368.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  habitable_planets['ESI (%)'] = habitable_planets.apply(compute_esi, axis=1)


In [ ]:
##graphics
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import shap
import os
os.makedirs("graphics", exist_ok=True)

for feature in features:
    plt.figure()
    sns.histplot(data=g, x=feature, hue='potentially_habitable', kde=True, palette='Set2')
    plt.title(f"{feature.capitalize()} Distribution by Habitability")
    if feature == 'flux':
        # Use more bins and no KDE for better accuracy in skewed data
        sns.histplot(data=g, x=feature, hue='potentially_habitable', kde=False, bins=60, palette='Set2')
        plt.xlim(0, 1000)  # or 0–10 if you want to focus only on the habitable range
    else:
        sns.histplot(data=g, x=feature, hue='potentially_habitable', kde=True, palette='Set2')

    plt.tight_layout()
    plt.savefig(f"graphics/{feature}_distribution_hist.png", dpi=300)
    plt.clf()

# Scatter Plot: Radius vs Temperature
sns.scatterplot(data=g, x='radius', y='temperature', hue='potentially_habitable', palette='coolwarm')
plt.title("Habitability Zone: Radius vs Temperature")
plt.xlabel("Radius (Earth radii)")
plt.ylabel("Equilibrium Temperature (K)")
plt.tight_layout()
plt.savefig("graphics/radius_vs_temperature.png", dpi=300)
plt.clf()

# Feature Importance Bar Plot
importances = model.feature_importances_
plt.barh(features, importances, color='teal')
plt.xlabel("Importance")
plt.title("Feature Importance (Random Forest)")
plt.tight_layout()
plt.savefig("graphics/feature_importance.png", dpi=300)
plt.clf()

# SHAP Explainability
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_new)

# X_new to DataFrame for SHAP
X_df = pd.DataFrame(X_new, columns=features)

if isinstance(shap_values, list):
    shap_to_plot = shap_values[1] if len(shap_values) > 1 else shap_values[0]
else:
    shap_to_plot = shap_values

if shap_to_plot.shape[0] != X_df.shape[0]:
    min_len = min(shap_to_plot.shape[0], X_df.shape[0])
    shap_to_plot = shap_to_plot[:min_len]
    X_df = X_df.iloc[:min_len]

# SHAP bar plot
shap.summary_plot(shap_to_plot, X_df, plot_type="bar", show=False)
plt.tight_layout()
plt.savefig("graphics/shap_feature_importance.png", dpi=300)
plt.clf()

# SHAP beeswarm
shap.summary_plot(shap_to_plot, X_df, show=False)
plt.tight_layout()
plt.savefig("graphics/shap_summary.png", dpi=300)
plt.clf()

print("📊 All visuals saved in /graphics folder for your poster.")

📊 All visuals saved in /graphics folder for your poster.


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 1150x660 with 0 Axes>

<Figure size 1150x660 with 0 Axes>